In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import mysql.connector
import traceback

def recommend_camp_sites(user_contents, num):
    ret = None
    conn = None
    try:
        cfg = {
            "host": "35.229.201.139",
            "port": 3306,
            "database": "camp_db",
            "user": "iii",
            "password": "Qwer_0987"
        }
        conn = mysql.connector.connect(**cfg)
        df = pd.read_sql("select * from camp_list", conn)
        train_series = df["tags"] # Series
        test_series = pd.Series(user_contents) # Series
#         tokenizer = TfidfVectorizer(analyzer="word", ngram_range=(1, 2), min_df=0, stop_words="english")
        tokenizer = TfidfVectorizer()
        train_tfidf = tokenizer.fit_transform(train_series)
        test_tfidf = tokenizer.transform(test_series)
        df_cs = pd.DataFrame(cosine_similarity(test_tfidf, train_tfidf), columns=train_series.index)
        df_cs["ix_test"] = test_series.index
        # pd.melt(): http://shzhangji.com/cnblogs/2017/09/30/pandas-and-tidy-data/
        # frame: 需要處理的數據框；
        # id_vars: 保持原樣的數據列；
        # value_vars: 需要被轉換成變量值的數據列；
        # var_name: 轉換後變量的列名；
        # value_name: 數值變量的列名。
        score = pd.melt(
            df_cs,
            id_vars = "ix_test",
            var_name = "ix_train",
            value_name = "score"
        )
        df["score"] = score["score"]
        # score遞增排序並取出指定的筆數
#         df = df.sort_values(by="score")[:-(num+1):-1]
        df = df.sort_values(by=["score", "fb_rating"], ascending=[False, False])
        ret = df[["camp_site", "tags", "score", "fb_rating"]].head(num)
    except Exception as e:
        print("Error: {}".format(e))
        traceback.print_exc()
    finally:
        if conn:
            conn.close()
    return ret
    

# recommend_result = recommend_camp_sites(user_contents="釣魚 螢火蟲 步道 美景", num=5)
recommend_result = recommend_camp_sites(user_contents="螢火蟲 步道", num=5)
print(recommend_result)

     camp_site       tags     score  fb_rating
366   李白梵谷露露山丘     步道 螢火蟲  1.000000       0.49
267   龍之泉休閒露營區     步道 螢火蟲  1.000000       0.00
345       達緄的家     步道 螢火蟲  1.000000       0.00
1004      羅馬之戀  水池 步道 螢火蟲  0.839741       0.66
863       團圓營地        螢火蟲  0.799213       0.66
